In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize  
from nltk.tokenize import WordPunctTokenizer 
from tensorflow.keras.preprocessing.text import text_to_word_sequence 
from math import log
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [31]:
nawon = pd.read_excel("1_내원정보.xlsx")


In [32]:
vatang = pd.read_excel("2_바탕질환.xlsx")
vatang

,VTHOS_NO,RGNO,ICD10CD,DIAG_NM,MAIN_DIAG_YN,ESB_TX_ID,ESB_STATE_CD,ESB_TIME
0,0xa08a0f54814fdb94e623cfe7d0cf992c6b42410388f5...,0xfb909db1b5b566ad3c87f28fab8eafd2bd9abbbb8ca7...,J21.9,Bronchiolitis,Y,NaN,NaN,NaN
1,0xa08a0f54814fdb94e623cfe7d0cf992c6b42410388f5...,0xfb909db1b5b566ad3c87f28fab8eafd2bd9abbbb8ca7...,A49.3,Mycoplasma infection,N,NaN,NaN,NaN
2,0xa08a0f54814fdb94e623cfe7d0cf992c6b42410388f5...,0xfb909db1b5b566ad3c87f28fab8eafd2bd9abbbb8ca7...,J18.9,Pneumonia,N,NaN,NaN,NaN
3,0x05635cc56defcb7f1772d06accb9569e9daf375025e4...,0xb5c92424b9a2980596af2742da4748adda60360ef32b...,J45.9,Asthma,Y,NaN,NaN,NaN
4,0xeb2054397645832f85b81f005333ee48d557b2460d3d...,0x47d02da7703550966d20712d43450f92bb417a1dc940...,T81.0,S/P tonsillectomy bleeding,Y,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
458256,0x0ac357d0cc0bc1df300267323ce3116ccee10e1dd166...,0x2e18a609f4f8bcf79d17ec375a5188685d4e286846bb...,B18.19,"Hepatitis B, viral, chronic",N,NaN,NaN,NaN
458257,0x0ac357d0cc0bc1df300267323ce3116ccee10e1dd166...,0x2e18a609f4f8bcf79d17ec375a5188685d4e286846bb...,K81.0,Empyema of gallbladder,N,NaN,NaN,NaN
458258,0x0ac357d0cc0bc1df300267323ce3116ccee10e1dd166...,0x2e18a609f4f8bcf79d17ec375a5188685d4e286846bb...,E87.2,Acidosis,N,NaN,NaN,NaN
458259,0x0ac357d0cc0bc1df300267323ce3116ccee10e1dd166...,0x2e18a609f4f8bcf79d17ec375a5188685d4e286846bb...,Z11.5,Special screening examination for Coronavirus ...,N,NaN,NaN,NaN


In [33]:
nawon

,VTHOS_NO,RGNO,AGRDE,SEX_CD,AREA_ADRES,VTHOS_CLSF_CD,IADM_DEPTT_NM,DEPTT_NM,ADM_YMD,LTHT_YMD,ESB_TX_ID,ESB_STATE_CD,ESB_TIME
0,0xc4ce7d5f0319782bc6ba2a198257d3fb1d45683a5631...,0xb737bf38f0b08920cedff6bf8a19ff7dbbc00b741a53...,50대,M,경기도 용인시,I,OS,OS,21060125,21060126,NaN,NaN,NaN
1,0xf6a8ce3aeed28eb76d4d3eff900c7af1bf911b52575e...,0x03a4abb30036750fe6170dc789e7f06c7279cd75c7a6...,70대,F,대전광역시 대덕구,I,NS,NS,20560929,20561004,NaN,NaN,NaN
2,0xa6a2a384d57ff6513821f71fcd4115721349983fe89b...,0x86c8a11b8d6011dbf005ee6e39e402010916bcbc189b...,50대,M,경기도 용인시,E,EM,EM,20650506,20650506,NaN,NaN,NaN
3,0x0c83e93507b7d9e420aad909a38cf858bbe2305a48e5...,0x66857189f4134314288f09beb96fd4b6dc68ee8452a0...,50대,M,경기도 용인시,I,OS,OS,21011202,21011211,NaN,NaN,NaN
4,0xbf6dd9b26eeedef81ec64430f12930e8f80a94db5cbd...,0x3ff79d3bb594b3e32bb606495906df2058efeb5b6e37...,20대,M,경기도 용인시,I,G4,G4,21060928,21060930,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249614,0x02603261cedef1f5b9e7921cf2589a37e19bf0618ad1...,0x36e7bee97402086df10ae76b03fe0152e462f7bde5fa...,20대,M,경기도 용인시,E,EM,EM,20540303,20540303,NaN,NaN,NaN
249615,0x59030044f210a45480c314dd8c320ff524710c6c01e9...,0xd8cb9aaa5a1caea1fe8f4961204223dd3af4e1b0c96a...,20대,F,경기도 용인시,E,EM,EM,20541214,20541214,NaN,NaN,NaN
249616,0xaad77dca956f38317f7a38f52473357f1ad5342c8438...,0xa92bbad07e975b32b0bf32ebd803ff6d911702be934d...,40대,M,서울특별시 구로구,I,UR,UR,20710730,20710804,NaN,NaN,NaN
249617,0x4689a79f4652d26bf4456f6e802c2b03fb710a7c18af...,0x06771025eed31fdeade626e80a7aa4efa7ec93bc2d50...,60대,M,경기도 용인시,I,M1,M1,20210317,20210321,NaN,NaN,NaN


In [34]:
nawon = nawon.drop(['ESB_TX_ID', 'ESB_STATE_CD', 'ESB_TIME'], axis=1)
nawon

,VTHOS_NO,RGNO,AGRDE,SEX_CD,AREA_ADRES,VTHOS_CLSF_CD,IADM_DEPTT_NM,DEPTT_NM,ADM_YMD,LTHT_YMD
0,0xc4ce7d5f0319782bc6ba2a198257d3fb1d45683a5631...,0xb737bf38f0b08920cedff6bf8a19ff7dbbc00b741a53...,50대,M,경기도 용인시,I,OS,OS,21060125,21060126
1,0xf6a8ce3aeed28eb76d4d3eff900c7af1bf911b52575e...,0x03a4abb30036750fe6170dc789e7f06c7279cd75c7a6...,70대,F,대전광역시 대덕구,I,NS,NS,20560929,20561004
2,0xa6a2a384d57ff6513821f71fcd4115721349983fe89b...,0x86c8a11b8d6011dbf005ee6e39e402010916bcbc189b...,50대,M,경기도 용인시,E,EM,EM,20650506,20650506
3,0x0c83e93507b7d9e420aad909a38cf858bbe2305a48e5...,0x66857189f4134314288f09beb96fd4b6dc68ee8452a0...,50대,M,경기도 용인시,I,OS,OS,21011202,21011211
4,0xbf6dd9b26eeedef81ec64430f12930e8f80a94db5cbd...,0x3ff79d3bb594b3e32bb606495906df2058efeb5b6e37...,20대,M,경기도 용인시,I,G4,G4,21060928,21060930
...,...,...,...,...,...,...,...,...,...,...
249614,0x02603261cedef1f5b9e7921cf2589a37e19bf0618ad1...,0x36e7bee97402086df10ae76b03fe0152e462f7bde5fa...,20대,M,경기도 용인시,E,EM,EM,20540303,20540303
249615,0x59030044f210a45480c314dd8c320ff524710c6c01e9...,0xd8cb9aaa5a1caea1fe8f4961204223dd3af4e1b0c96a...,20대,F,경기도 용인시,E,EM,EM,20541214,20541214
249616,0xaad77dca956f38317f7a38f52473357f1ad5342c8438...,0xa92bbad07e975b32b0bf32ebd803ff6d911702be934d...,40대,M,서울특별시 구로구,I,UR,UR,20710730,20710804
249617,0x4689a79f4652d26bf4456f6e802c2b03fb710a7c18af...,0x06771025eed31fdeade626e80a7aa4efa7ec93bc2d50...,60대,M,경기도 용인시,I,M1,M1,20210317,20210321


In [35]:
nawon['ADM_YMD'] = nawon['ADM_YMD'].map(lambda x : (x//1000000)*10000+(x%10000))

0         210125
1         200929
2         200506
3         211202
4         210928
           ...  
249614    200303
249615    201214
249616    200730
249617    200317
249618    200522
Name: ADM_YMD, Length: 249619, dtype: int64

In [36]:
nawon['LTHT_YMD'] = nawon['LTHT_YMD'].map(lambda x : (x//1000000)*10000+(x%10000))

In [43]:
nawon['AGRDE'].unique()

array(['50대', '70대', '20대', '40대', '10대미만', '10대', '30대', '60대', '80대',
       '90대', '100세 이상'], dtype=object)

In [54]:
nawon[nawon['RGNO']=='0xd4ca321f29db6a03e62862ff1037227fc6d019bebead791abd4cc550ce22b4b7']

,VTHOS_NO,RGNO,AGRDE,SEX_CD,AREA_ADRES,VTHOS_CLSF_CD,IADM_DEPTT_NM,DEPTT_NM,ADM_YMD,LTHT_YMD
67574,0x288a5b75428b81128ab4fefad610791385e1368b2574...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,80대,M,경기도 용인시,I,M1,M1,200101,200115
67575,0x4d67849c91af6a165461f9fd8951368082017646b35a...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,80대,M,경기도 용인시,I,M1,M1,201106,201111
67576,0x3a47717289a7813d9c5646a2ffb9c4a4aca26a80d37c...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,80대,M,경기도 용인시,I,M1,M1,210729,210729
67577,0x6fa46a045ed8f462606b1c0cb6f27d488b890b5e076d...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,80대,M,경기도 용인시,I,M1,M1,200113,200113
67578,0xf97474f3db4dc56dd67a0c6c6bae880eab214a94238f...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,80대,M,경기도 용인시,I,M1,M1,201109,201109
67579,0x4713631fdcfc16e0f9c774e52f1fa861a6b782cb7e40...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,80대,M,경기도 용인시,I,M1,M1,201212,201212
67580,0x1f9e68326917be6dbafa5f4a117d1f1ebbd333d419c3...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,80대,M,경기도 용인시,I,M1,M1,201031,201114
67581,0x051147b74e9d19c05c5d9ea44f8e8310c364dd6b6067...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,80대,M,경기도 용인시,I,M1,M1,200811,200816
67582,0xe803db9cf28b7f455e8ca5673a65278ee1e747a94084...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,80대,M,경기도 용인시,I,M1,M1,200601,200601
67583,0xaf3b0effa3eb2cffd95b3eff6d5e708fc4be159c61ac...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,80대,M,경기도 용인시,I,M1,M1,200315,200319


In [47]:
nawon['RGNO'].value_counts()

0x089d1b95812d443dd43068f13b792f4511cb322c5606e8f90d60d9f812bc518f    64
0xdad6fd1e94e9185c1dbe951794319de9f70b1bf14a2fa218a6f4aeb58ca1f48f    55
0xac03f9803ec7f0d9cf547e3eb8ca8801196327398cb4c0bdefd84718c79aebb9    53
0xd4ca321f29db6a03e62862ff1037227fc6d019bebead791abd4cc550ce22b4b7    50
0x63a7fc2d346926fdc1e86febbafcf02afaf67e1e5bfd515084edca19c7d7fc67    47
                                                                      ..
0x70796d5f8c9a877a5c364ddc97817579ce4e5b6345914272c85e92f015560ab9     1
0x0bcd724833aafc33579f64c3f4349386535738ca0fede859cb04e7ed0721da84     1
0x47c99248f145eabaf53db319e9e09c244d488c28553d5fe8b31ee3ed4589b328     1
0x06f01380938ff301736cdab3a39dffe0542740c62585068d6739ec0451ee1a89     1
0xc86007d9d665d773988a8d78f9032618a109f043e2536b6ec7941eac617aa244     1
Name: RGNO, Length: 147357, dtype: int64

In [55]:
vatang[vatang['RGNO']=='0xd4ca321f29db6a03e62862ff1037227fc6d019bebead791abd4cc550ce22b4b7']

,VTHOS_NO,RGNO,ICD10CD,DIAG_NM,MAIN_DIAG_YN,ESB_TX_ID,ESB_STATE_CD,ESB_TIME
47570,0x288a5b75428b81128ab4fefad610791385e1368b2574...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,C24.0,"Klatskin's, Bismuth type IV",Y,NaN,NaN,NaN
47571,0x288a5b75428b81128ab4fefad610791385e1368b2574...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,K83.1,Obstructive jaundice,N,NaN,NaN,NaN
47572,0x288a5b75428b81128ab4fefad610791385e1368b2574...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,K85.9,Acute pancreatitis,N,NaN,NaN,NaN
47573,0x288a5b75428b81128ab4fefad610791385e1368b2574...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,K26.9,Duodenal ulcer,N,NaN,NaN,NaN
54664,0x4d67849c91af6a165461f9fd8951368082017646b35a...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,C24.0,"Klatskin's, Bismuth type IV",Y,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
99646,0x051147b74e9d19c05c5d9ea44f8e8310c364dd6b6067...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,C24.0,"Klatskin's tumor, Bismuth type IV",N,NaN,NaN,NaN
99647,0x051147b74e9d19c05c5d9ea44f8e8310c364dd6b6067...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,K75.9,Hepatitis,N,NaN,NaN,NaN
99648,0x051147b74e9d19c05c5d9ea44f8e8310c364dd6b6067...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,C79.9,Carcinomatosis,N,NaN,NaN,NaN
99649,0x051147b74e9d19c05c5d9ea44f8e8310c364dd6b6067...,0xd4ca321f29db6a03e62862ff1037227fc6d019bebead...,K21.9,Gastro-esophageal reflux disease,N,NaN,NaN,NaN
